In [ ]:
from __future__ import annotations
import numpy as np
import random
import copy
import importlib

from typing import Tuple, List
from numpy import array, zeros
from matplotlib import pyplot as plt

# from Big_Class import Big_Class  # already imported one NETfuncs is imported
from User_Variables import User_Variables  # already imported one NETfuncs is imported
from Network_Structure import Network_Structure  # already imported one NETfuncs is imported
from Big_Class import Big_Class
from Network_State import Network_State
from Networkx_Net import Networkx_Net
import matrix_functions, functions, statistics, colors, plot_functions

## Colors

In [ ]:
colors_lst, red, cmap = colors.color_scheme()
cmap

## Prelims

In [ ]:
## Parameters

Nin = 2
Nout = 3

## task type
task_type='Regression'

# extra nodes
extraNin: int = 0
Ninter: int = 0
extraNout: int = 0

# resistance-pressure proportionality factor
gamma: np.ndarray = np.array([1.0])

## method to update resistances - physical property of the system
R_update: str = 'R_propto_dp'
# R_update: str = 'deltaR_propto_dp'
# R_update: str = 'deltaR_propto_Q'
# R_update: str = 'deltaR_propto_Power'
R_vec_i = array([1.])

stay_sample: int = 2

# length of training dataset
iterations = int(2e4)  # number of sampled of p

stay_sample = 2

# measure accuracy every # steps
measure_accuracy_every = 15

supress_prints: bool = True  # whether to print information during training or not
bc_noise: float = 0.0  # noise to dual problem
use_p_tag: bool = False  # use 1 or 2 sampled pressures at every time step
include_Power: bool = False
lam: float = 0
    
access_interNodes: bool = False  # access and change pressure at interNodes (nodes between input and output) or not
noise_to_extra: bool = False  # add noise to extra outputs 
    
window_for_mean = 1000
    
alpha: float = 0.2  # for network combine attempt

## funcs

In [ ]:
def network_build_given_Nin_Nout(Nin: int, Nout: int, M_norm: NDArray[np.float_]) -> tuple():
    
    # initialize Variables
    Variabs = User_Variables(dataset_size,\
                             Nin, \
                             extraNin, \
                             Ninter, \
                             Nout, \
                             extraNout, \
                             gamma, \
                             R_update, \
                             use_p_tag, \
                             include_Power, lam, \
                             supress_prints, \
                             bc_noise, \
                             access_interNodes, \
                             task_type, \
                             measure_accuracy_every)
    
    Variabs.assign_alpha_vec(alpha)
    Variabs.create_dataset_and_targets(rand_state, M_norm[:])
    Variabs.create_noise_for_extras()
    BigClass = Big_Class(Variabs)
        
    # initialize Structure
    inInterOutGround_tuple = matrix_functions.build_input_output_and_ground(Variabs.Nin, Variabs.extraNin, Variabs.Ninter, 
                                                                            Variabs.Nout, Variabs.extraNout)
        
    
    Strctr = Network_Structure(inInterOutGround_tuple)
    Strctr.build_incidence()
    Strctr.build_edges()
    BigClass.add_Strctr(Strctr)  # add to big class
    
    # initialize State    
    State = Network_State(Variabs)
    State.initiate_resistances(BigClass, R_vec_i)
    State.initiate_accuracy_vec(BigClass, measure_accuracy_every)
    BigClass.add_State(State)  # add to big class
    
    return Variabs, Strctr, State, BigClass


def train_loop(Variabs, Strctr, State, BigClass):
    loss_mean = [1, 1]
    for l in range(iterations):
        
        k = (l//stay_sample)*2 + l%2

        # draw input and desired outputs from dataset
        if not((l+1) % 4):  # add noise only at i=3 etc.
            State.draw_p_in_and_desired(Variabs, k, noise_to_extra=False)  # add noise to extra nodes every 2nd iteration
            State.solve_flow_given_problem(BigClass, "measure", noise_to_extra=False)  # measure and don't change resistances
        else:  # dont add noise to extra nodes
            State.draw_p_in_and_desired(Variabs, k)
            State.solve_flow_given_problem(BigClass, "measure")

        if not l % 2:  # even iterations, take another sampled pressure and measure again
            pass
        else:  # odd iterations, go to dual problem and update resistances
            State.t += 1
            State.calc_loss(BigClass)
            State.update_input_dual(BigClass)
            State.update_output_dual(BigClass)
            State.solve_flow_given_problem(BigClass, "dual", access_inters=False)  # measure and don't change resistances
            State.update_Rs(BigClass)
            
        if not (l+1)%window_for_mean:
            # print('l', l)
            loss_mean.append(np.mean(np.mean(np.abs(State.loss_norm_in_t[-window_for_mean:]), axis=1)))
            # print(loss_mean)
            
        if loss_mean[-1]<10e-10 or (loss_mean[-1]-loss_mean[-2])>0 or loss_mean[-1]>2:
            break
            
    return State

def plot_bars_Mvals(M_norm, M_pinv, M_networksol):
    # Flatten the matrices to get individual entries
    M_norm_flat = M_norm.flatten()
    M_pinv_flat = M_pinv.flatten()
    M_networksol_flat = M_networksol.flatten()

    # Create a figure and axis
    fig, ax = plt.subplots()

    # The number of groups
    n_vals = np.size(M_norm_flat)

    # Bar width
    bar_width = 0.2

    # X locations for the groups
    index = np.arange(n_vals)

    # Plot bars for each matrix, offsetting them slightly for grouped appearance
    bar1 = ax.bar(index - bar_width, M_pinv_flat, bar_width, label='M psuedo inv', color=colors_lst[0])
    bar2 = ax.bar(index, M_norm_flat, bar_width, label='M Normalized', color=colors_lst[1])
    bar3 = ax.bar(index + bar_width, M_networksol_flat, bar_width, label='M Network Solution', color=colors_lst[2])

    # Add labels, title, and axes ticks
    ax.set_ylabel('M Values')
    ax.set_xticks(index)

    # Add a legend
    ax.legend()

    # Display the plot
    plt.show()

## M

In [ ]:
rand_state=array([43])

M_values = matrix_functions.random_gen_M(rand_state[0], Nout*Nin)
M = M_values[0:Nout*Nin].reshape(Nout, Nin)
M_line = np.sum(M, axis=1)
M_norm = M[:Nin*Nout]/np.max(M_line)*0.75  # normalize so max sum over line will be 0.75

print('M_norm')
print(M_norm)

## X

In [ ]:
X_values = matrix_functions.random_gen_M(rand_state, Nin*Nin)  # take random samples of pressure
X = X_values.reshape(Nin, Nin)

## Multiple redundancies

In [ ]:
pinv_goodness_vec = zeros(Nin)
network_goodness_vec = zeros(Nin)

for i in range(Nin):
    redundancy = i
    dataset_size = np.shape(X)[0]-redundancy  # change num of samples accordingly
    
    # handicap X
    for j in range(redundancy):
        X[j+1,:] = X[0,:]  # last sample is the same as one before, overdetermined set of equations

    # desired
    Y = np.matmul(X,M_norm.T)  # sized NoutxNin
    
    # Pseudo Inverse
    M_pinv = np.matmul(np.linalg.pinv(X),Y).T
    
    # Network
    Variabs, Strctr, State, BigClass = network_build_given_Nin_Nout(Nin, Nout, M_norm.ravel())
    State = train_loop(Variabs, Strctr, State, BigClass)
    plot_functions.plot_importants(State, Variabs, State.desired_in_t, Variabs.M, include_network=True)
    
    # extract solved M using only a single input of 1 at every time
    M_networksol = zeros([Nout, Nin])
    for j in range(Nin):
        State.input_drawn = zeros(Nin)
        State.input_drawn[j] = 1
        State.solve_flow_given_problem(BigClass, "measure", noise_to_extra=False)
        M_networksol[:,j] = State.output
        
    plot_bars_Mvals(M_norm, M_pinv, M_networksol)
    
    pinv_goodness = np.mean((M_norm.flatten() - M_pinv.flatten()) ** 2)/np.mean((M_norm.flatten()) ** 2)
    network_goodness = np.mean((M_norm.flatten() - M_networksol.flatten()) ** 2)/np.mean((M_norm.flatten()) ** 2)

    print('pinv goodness', pinv_goodness)
    print('network goodness', network_goodness)
    
    pinv_goodness_vec[i] = pinv_goodness
    network_goodness_vec[i] = network_goodness

In [ ]:
plt.plot(Nin-np.arange(Nin), pinv_goodness_vec, color=colors_lst[0])
plt.plot(Nin-np.arange(Nin), network_goodness_vec, color=colors_lst[1])
plt.legend(['pseudo inverse', 'network'])
plt.ylabel('MSE badness')
plt.xlabel('dataset size')
plt.show()

## Desired

In [ ]:
Y = np.matmul(X,M_norm.T)  # sized NoutxNin

print(Y)

## Pseudo inv

finding M from inputs X and deisred outputs Y where the last two lines of X are the same, so overdetermined set of equations

In [ ]:
M_pinv = np.matmul(np.linalg.pinv(X),Y).T

print('M pseudo inv')
print(M_pinv)

## Network solution

In [ ]:
# solve the network
Variabs, Strctr, State, BigClass = network_build_given_Nin_Nout(Nin, Nout, M_norm.ravel())
State = train_loop(Variabs, Strctr, State, BigClass)

In [ ]:
plot_functions.plot_importants(State, Variabs, State.desired_in_t, Variabs.M, include_network=True)

In [ ]:
# extract solved M using only a single input of 1 at every time
M_networksol = zeros([Nout, Nin])
for i in range(Nin):
    State.input_drawn = zeros(Nin)
    State.input_drawn[i] = 1
    State.solve_flow_given_problem(BigClass, "measure", noise_to_extra=False)
    M_networksol[:,i] = State.output

In [ ]:
State.R_in_t[0]

In [ ]:
plt.plot(np.mean(np.mean(np.abs(State.loss_norm_in_t), axis=1), axis=1))
plt.ylabel('Loss')
plt.xlabel(r'$t$')

In [ ]:
print('M (norm)')
print(M_norm)

print('M pseudo inv')
print(M_pinv)

print('M network solution')
print(M_networksol)

In [ ]:
# Flatten the matrices to get individual entries
M_norm_flat = M_norm.flatten()
M_pinv_flat = M_pinv.flatten()
M_networksol_flat = M_networksol.flatten()

# Create a figure and axis
fig, ax = plt.subplots()

# The number of groups
n_vals = np.size(M_norm_flat)

# Bar width
bar_width = 0.2

# X locations for the groups
index = np.arange(n_vals)

# Plot bars for each matrix, offsetting them slightly for grouped appearance
bar1 = ax.bar(index - bar_width, M_pinv_flat, bar_width, label='M psuedo inv', color=colors_lst[0])
bar2 = ax.bar(index, M_norm_flat, bar_width, label='M Normalized', color=colors_lst[1])
bar3 = ax.bar(index + bar_width, M_networksol_flat, bar_width, label='M Network Solution', color=colors_lst[2])

# Add labels, title, and axes ticks
ax.set_ylabel('M Values')
ax.set_xticks(index)

# Add a legend
ax.legend()

# Display the plot
plt.show()

In [ ]:
pinv_goodness = np.mean((M_norm_flat - M_pinv_flat) ** 2)/np.mean((M_norm_flat) ** 2)
network_goodness = np.mean((M_norm_flat - M_networksol_flat) ** 2)/np.mean((M_norm_flat) ** 2)

print('pinv goodness', pinv_goodness)
print('network goodness', network_goodness)